In [1]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(800, 1, 2, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 520/520 [00:40<00:00, 12.92it/s]


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [2]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain[:4]))]
n_cpu = 4
res_list = run_pll(run_autognet, args, processes=n_cpu)
res_list_array = np.array(res_list)


Multiprocessing <function run_autognet at 0x1693ded40> in 4 tasks, with 4 processes...
psi_zero: 0.45002499999999995
psi_1_gamma: 0.25947499999999996
psi_0_gamma: 0.4129
average: 0.31045
direct_effect: -0.15342500000000003
spillover_effect: -0.037124999999999964
psi_zero: 0.477
psi_1_gamma: 0.2489
psi_0_gamma: 0.467725
average: 0.31842499999999996
direct_effect: -0.218825
spillover_effect: -0.009274999999999978
psi_zero: 0.48885
psi_1_gamma: 0.281525
psi_0_gamma: 0.501375
average: 0.35052500000000003
direct_effect: -0.21985
spillover_effect: 0.012525000000000008
psi_zero: 0.5075
psi_1_gamma: 0.2733499999999999
psi_0_gamma: 0.43705
average: 0.327025
direct_effect: -0.16370000000000007
spillover_effect: -0.07044999999999996
Multiprocessing finished.


In [3]:
from run_pll import column_names

ret_mean = res_list_array.mean(axis=0)
ret_std = res_list_array.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean[i]:.5f} ± {ret_std[i]:.5f}")

average: 0.32661 ± 0.01500
direct_effect: -0.18895 ± 0.03061
spillover_effect: -0.02608 ± 0.03108
psi_0_gamma: 0.45476 ± 0.03319
psi_zero: 0.48084 ± 0.02085
psi_1_gamma: 0.26581 ± 0.01255


In [ ]:
{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [3]:
from autognet import evaluate_autognet_via_agc_effect

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}
for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autognet_via_agc_effect(adj, Y, A, L, treatment_allocation=0.7, R=50, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))

1it [00:21, 21.48s/it]

psi_zero: 0.5075
psi_1_gamma: 0.2733499999999999
psi_0_gamma: 0.43705
average: 0.327025
direct_effect: -0.16370000000000007
spillover_effect: -0.07044999999999996


2it [00:43, 21.77s/it]

psi_zero: 0.45002499999999995
psi_1_gamma: 0.25947499999999996
psi_0_gamma: 0.4129
average: 0.31045
direct_effect: -0.15342500000000003
spillover_effect: -0.037124999999999964


3it [01:02, 20.42s/it]

...

psi_zero: 0.436825
psi_1_gamma: 0.27194999999999997
psi_0_gamma: 0.40025
average: 0.31425000000000003
direct_effect: -0.12830000000000003
spillover_effect: -0.036575000000000024


27it [08:44, 18.24s/it]

psi_zero: 0.4925
psi_1_gamma: 0.283975
psi_0_gamma: 0.4667
average: 0.343925
direct_effect: -0.18272500000000003
spillover_effect: -0.02579999999999999


27it [08:49, 19.61s/it]


KeyboardInterrupt: 

## Dense network

In [ ]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(800, 6, 8, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

In [ ]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

In [ ]:
from autognet import evaluate_autognet_via_agc_effect

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}
for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = evaluate_autognet_via_agc_effect(adj, Y, A, L, treatment_allocation=0.7, R=50, burnin=10, seed=1)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))